# First steps in PySpark 

In this notebook we will learn the fundamentals of functional programming, as well as the basic abstraction of a distributed object in Spark, the RDD. The notebook has been divided into two parts:

Part 1: map/reduce basics

Part 2: Work with RDD and Pair RDD abstractions 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

# Part 1: map/reduce basics

![Hadoop Logo](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Hadoop_logo.svg/220px-Hadoop_logo.svg.png)
# **Apache Hadoop (MapReduce)**

It is an open source software framework written in Java for distributed storage and distributed processing of very large data sets on computer clusters built from commodity hardware. All the modules in Hadoop are designed with a fundamental assumption that hardware failures (of individual machines, or racks of machines) are common and thus should be automatically handled in software by the framework.

The core of Apache Hadoop consists of a storage part (Hadoop Distributed File System (HDFS)) and a processing part (MapReduce). Hadoop splits files into large blocks and distributes them amongst the nodes in the cluster. To process the data, Hadoop MapReduce transfers packaged code for nodes to process in parallel, based on the data each node needs to process. This approach takes advantage of data locality — nodes manipulating the data that they have on hand — to allow the data to be processed faster and more efficiently than it would be in a more conventional supercomputer architecture that relies on a parallel file system where computation and data are connected via high-speed networking.

![caption](http://d152j5tfobgaot.cloudfront.net/wp-content/uploads/2012/07/mapreduce.png)

Since data and computation are distributed, we should avoid the use of variables, i.e. mutable data. Thus, in contrast to impertaive programming, we shall use the functional approach (lambda calculus).

### The goal of the following excercises is to understand basic lambda calculus with python.

### (1a) Functional programming in Python

So, what is Functional Programming? From Wikipedia: 

« …a  programing paradigm that treats computation as the evaluation of mathematical functions and **avoids changing-state and mutable  data**.»

It´s based upon Lambda calculus, wich consist of:
 * Function definition (declaration of expressions)
 * Function application (evaluation of those expressions)
 * Recursion (iteration)

We have already used this in python!!! :)

Recall the typical "lambda x: x+1" we have been using as the first argument of map, reduce and filter methods:
 * **map** maps each value in the input collection to a different value. It´s just the classical mathematical funciton we are used to!
 * **reduce** takes two values from the input collection and returns a new value (of the same type) by appliying a commutative operation to them. 
 * **filter** filters the elements in the input collection according to a certain (boolean) criteria.
 

**Mapping**

`map` is a Higher Order Function (HOF) that takes a function f and a sequence and returns a new sequence formed by applying f to each element in the original sequence. 

![map](https://cosminpupaza.files.wordpress.com/2015/10/map.png?w=505)

Very often we write lambdas, anonymous functions, to use in defining simple transformations with `map`:


In [1]:
#Ejemplo para hacer el cuadrado
input_list=[1,7,98]
map(lambda n:n**2,input_list)

In [2]:
#Para que podamos ver el resultado:
list(map(lambda n:n**2,input_list))

[1, 49, 9604]

In [3]:
#ejemplo para formar plurales
list(map(lambda string: string + 's',['coche','moto']))

['coches', 'motos']

In [4]:
#ejemplo con def en vez de con funcoines lambda

def pluralize(word):
    return word+'s'

list(map(pluralize,['coche','moto']))

['coches', 'motos']

We can also use named functions, of course. Note that we pass them as arguments to `map`, not execute them. You can think of them as tools we give to `map` for it to use.

Consider the following very common pattern: create empty list, append transformed elements one by one:

In [5]:
plurals=[]

for element in ['coche','moto']:
    plurals.append(pluralize(element))

plurals

#Esto es básicamente lo que hace un map, en este caso en concreto.

['coches', 'motos']

We can abstract over every possible list by defining a function that takes the list as an argument:

In [8]:
#generalizando para cualquier lista

def pluralize_list(input_list):
    plurals=[]
    
    for element in input_list:
        plurals.append(pluralize(element))

    return plurals

pluralize_list(['coche','moto'])

['coches', 'motos']

Consider this other piece of code: we are doing a very similar thing, but with a different function. A lot of the code is common between the two little programs.

Wouldn't it be convenient if we could abstract over every function, as we did over every list?

In [10]:
#abstrayendo más aún (más versatil, pq imaginar que este patron es super super común):

def initials(input_list):
    plurals=[]
    
    for element in input_list:
        plurals.append(element[0])

    return plurals

initials(['coche','moto'])


['c', 'm']

We can, taking the function to be applied as an argument. This is because Python treats functions as first-class objects, so they can be used as arguments to other functions.

In [13]:
#queremos aplicar distintas transformaciones del mismo tipo. Convertimos la transformacion que queremos aplicar en argumentos. 
#Paso un argumento. y desde dentro de mi funcion aplico la funcion f. ¿qué funcion es f? Puede ser cualquiera, solamente tengo que pasarla
#como argumento. Esto ya es una funcion de orden superior, coge una lista y le aplica una funcion, eemento a elemento.

def process_list(f,input_list):
    plurals=[]
    
    for element in input_list:
        plurals.append(f(element))

    return plurals

process_list(pluralize,['coche','moto'])

['coches', 'motos']

That function we just wrote, `process_list`, is exactly what `map` is

In [14]:
list(map(lambda string:string[0], ['coche','moto']))

['c', 'm']

In [20]:
#Un apply a un dataframe es exactamente esto. 

absurd_animals=['platypus','zebra','giraffe','naked mole rat']
sorted(absurd_animals)

#con sorted ordenamos de forma alfabéticamente. Pero que pasa si queremos ordenarlo en base a la segunda letra. 
#Ya no es razonable esperar que haya un argumento como reverse. 
# Tenemos el argumento 'key', que es un argumentoal al que se le asigna una funcion, que se aplicará a cada elemento de la lista.
sorted(absurd_animals, key=lambda word: word[1])

['naked mole rat', 'zebra', 'giraffe', 'platypus']

In [18]:
#según la longitud de la palabra:
sorted(absurd_animals,key=len)

['zebra', 'giraffe', 'platypus', 'naked mole rat']

**Filtering**

`filter` encodes another very common pattern: building a sequence from some of the elements of an input sequence, deciding whether to include each based on the result of evaluating a function, often called the `predicate`, on each of the elements.


![filter](https://cosminpupaza.files.wordpress.com/2015/11/filter.png?w=405)

In [21]:
input_list

[1, 7, 98]

In [22]:
#coger un único elemento y devolver un booleano. El booleano que se genere por cada elemento de la lista me indicará si quiero quedarme con ella o no.
list(filter(lambda n: n<10, input_list))

[1, 7]

Again, we can use either lambdas or named functions.

In [25]:
def predicate(n):
    return n<10

list(filter(predicate, input_list))

[1, 7]

#### Exercise

write a filter to extract those words that have more than 3 letters from `collection_of_strings`

In [26]:
collection_of_strings=['Holi','amigui','mii','vinti','a','mi','casita']

In [31]:
list(filter(lambda n: len(n)<3, collection_of_strings))

['a', 'mi']

#### Exercise

Write a filter to extract prime numbers from the following list of integers:

```python
[12, 17, 19, 18, 23, 24]
```

You will need to write a function that determines whether a single number is a prime or not.

In [33]:
exercise=[12, 17, 19, 18, 23, 24]

In [37]:
#funcion para ver si es primo:
def isPrime(n):
    for j in range(2,n):
        if n%j==0:
            return False

    return True

In [44]:
#Sacamos los primos de la lista que le pasemos.
list(filter(isPrime,exercise))

[17, 19, 23]

`map` and `filter` are not  used explicitly very often in Python, in part because of their cumbersome syntax. 

Nevertheless, there is a well-known feature in Python that is in fact just a convenient way to write `map`s and `filter`s: **list comprehensions**:

In [45]:
#list_comprenhension es programacion funcional disfrazada. 
#como sacamos los números primos de la lista con una list comprenhension:

[ isPrime(n) for n in exercise]

#esto da una lista de booleanos, pero no nos devuelve lo que queremos:

[False, True, True, False, True, False]

In [50]:
#una list comprenhension que emula lo anteior sería:
[ n for n in exercise if isPrime(n) ]

#Esta list_comprehension es realmente un filter()

#Con las list comprenhension podemos encadenar operaciones, por ejemplo un map y un filter a la vez.

[17, 19, 23]

In [51]:
#cuagrados de los 3 numeros primos, lo que es un filter seguido de un map
[ n**2 for n in exercise if isPrime(n) ]

[289, 361, 529]

In [52]:
#equivalente con maps y filter. Es una sintaxis un poco infame.
list(map(lambda n: n**2, filter(isPrime,exercise)))

[289, 361, 529]

The above list comprehension is exactly equivalent to:

In [ ]:
#correponde a la celda anterior

Which is a pipeline of two steps: first a `filter`, then a `map`

**Reducing** 

`reduce` is the third basic foundation of functional programming. Reduce uses a function, often called the `combiner`, to transform a sequence of elements of type T (that is, any type) into a single T. The combiner must take 2 Ts and return only one T.


Recall it must be commutative! Think about the importance of this when parallelizing computations

![reduce](https://cosminpupaza.files.wordpress.com/2015/11/reduce.png?w=500)

In [54]:
#hay que importar reduce
from functools import reduce

In [55]:
#reduce toma una funcion, el combiner y agregar dos elementos
#ejemplo sumatorio
reduce(lambda x,y: x+y, exercise)

113

In [56]:
#ejemplo productorio
reduce(lambda x,y: x*y, exercise)

38511936

#### Exercise

Write a function_x such that, when applied with `reduce`, that reduce returns the highest number in a list. Don't use the built-in `max`!

In [58]:
exercise

[12, 17, 19, 18, 23, 24]

In [65]:
#reduce(lambda x,y: max(x,y), exercise)

#algo equivalente definiendo nosotros una funcion sería:
def f_max(x,y):
    return x if x>y else y #esto es una sintaxis ternaria
    
    
reduce(f_max, exercise)

24

In [ ]:
#pero como lo hace cogiendo de 2 en 2?

#aplica a los dos primeros elementos de la lista, luego coge el resultado de la agregacion y compara este resultado y el segundo, y así hasta el final.

#Tenemos que intentar que todo pueda expresarse de esta forma.
#X=[x1, x2, x3, x4]
#si hago reduce(f,X), es equivalente a f(f(f(x1,x2),x3),x4)

#la f debe ser conmutativa, lo anterior debe ser igual a esto: f(f(x1,x2),f(x3,x4))

#Expresar una funcion en terminos de reduce() la hace arbitrariamente paralelizable.


## (1b) Exercise: Calculate the mean of a collection of real numbers using map/reduce
Recall:

$$\bar x = \frac{\sum_{i=1}^{N} x_i}{N} $$

It´s straightforward to do this with python built-in mehots sum() and len(). However, how would you do that with map/reduce? We have already shown how to sum the elements of an array. Thus, you have to calculate the length of the array. For this:
 * Create another array of the same size, consisting of 1s.
 * Sum the elements of that array

#### First part

* Do a reduce to do the sum, and a different map-reduce to get the length

In [70]:
total=reduce(lambda x,y: x+y, exercise)

In [68]:
#para sacar la longitud, hay que hacer una suma pero de 1s, transformando nuestro array en una lista de 1s y sumando
list(map(lambda x:1,exercise))

[1, 1, 1, 1, 1, 1]

In [71]:
#aquí está la longitud del vector
count=reduce(lambda x,y: x+y, map(lambda x:1,exercise))

That was also a pipeline of two steps. It's equivalent to:

#### Combine them in one pass

Think about what it would mean to finish one computation and only then do another in a distributed environment.

Combine both in one pass:

In [92]:
#lo primero es transformamos los n en tuplas n,1
def f(n): 
    return (n,1)

#saca suma
def combiner(tuple1,tuple2):
    runningtotal=tuple1[0]+tuple2[0]
    runningcount=tuple1[1]+tuple2[1]
    
    return (runningtotal,runningcount)
    
    
#tenemos que hacer un reduce que haga a la vez la suma y la cuenta.
#el reduce sumara dos nueros en paralelo, uno que sumará la suma total y el otro la suma de unos.
#cada tupla representa un resultado parcial, vendrá de un único número

In [93]:
total, count=reduce(combiner,map(f,exercise))

#empezar por el final, reduce, tiene que producir el total y el count, es decir una tupla de 2 elementos. EL combinier tiene que que comer entonces
#dos tuplas de dos elementos. La f tiene que comer un numero y delvolver una tupla.

In [94]:
print(total,count)
#la media es:
med=total/count
print(med)

113 6
18.833333333333332


## (1c) Exercise: Calculate the standard deviation of a collection of real numbers

We'll use the typical definition of the standard deviation:

$$\sigma_x^2 = \frac{\sum_{i=1}^{N} (x_i-\bar x)^2}{N-1}$$

For this, use the *mean* and *count* variables from the previous excercise.

In [87]:
sum_squared_differences=0
for number in exercise:
    sum_squared_differences+=(number-med)**2

varianza=sum_squared_differences/(count-1)
varianza

18.96666666666667

In standard idiomatic Python we'd do it like this:

With `map` and `reduce`:

In [102]:
#al final el map es una transformacion. Cada vez que veamos una sumatoria es un reduce.

list(map(lambda n: (n-med)**2,exercise))

[46.69444444444443,
 3.3611111111111067,
 0.027777777777778172,
 0.6944444444444424,
 17.36111111111112,
 26.694444444444457]

In [97]:
reduce(lambda x,y:x+y, map(lambda n: (n-med)**2,exercise))/(count-1)

18.96666666666667

#### Calculate the standard deviation in one pass.

Again, think about what it would mean to finish one computation and only then do another in a distributed environment.


If we can, we should avoid broadcasting variables all over the cluster. In this case, the following alternative definition of standard deviation can come in handy:

$$\sigma_x^2 = \frac{\sum_{i=1}^{N} (x_i-\bar x)^2}{N-1} =
\frac{\sum_{i=1}^{N} (x_i^2+{\bar x}^2-2x_i\bar x)}{N-1} =
\frac{1}{N-1}\left(\sum_i x_i^2-N\bar x^2\right)$$



In [105]:
#hemos tenido que esperar a que acabe el calculo del total y el count para poder calcular la varianza, y esta no es la idea.
#podemos hacerlo en paralelo todo re-escribiendo la formula.

#podemos hacer esto en un solo pase. Es lo mismo de antes, antes hemos hecho para calcular la media sacar la tupla (total,count)
#pues ahora hay que meter un término más a las tuplas

def f(n): 
    return (n**2,n,1)

#saca suma
def combiner(tuple1,tuple2):
    runningsumsq=tuple1[0]+tuple2[0]
    runningtotal=tuple1[1]+tuple2[1]
    runningcount=tuple1[2]+tuple2[2]
    
    return (runningsumsq,runningtotal,runningcount)


In [107]:
sum_squares, total, count=reduce(combiner,map(f,exercise))

(sum_squares-count*(total/count)**2)/(count-1)

18.966666666666697

#### Side note

In Python 2, map and filter returned lists. In Python 3, they return generators, which are lazy collections. They are somewhat similar to files in that they can be depleted of elements after iterating through them.

## (1d) Twe 'word-count' problem: creating histograms
Given a set of keys in an input collection, calculate the frequency of each key. 

In order to understand better how map/reduce works, we will implement this simple calculation in several forms.

We are going to use as test case a list of elements chosen from a small set:

In [ ]:
#esto es el hola mundo del mundo distribuido.
#el objetivo es dado una coleccion de palabas, calcular la frecuencia de cada palabra.

In [134]:
absurd_animals

import random
random.seed(42)

my_absurd_zoo=[random.choice(absurd_animals) for a in range(100)]
my_absurd_zoo

['platypus',
 'platypus',
 'giraffe',
 'zebra',
 'zebra',
 'zebra',
 'platypus',
 'platypus',
 'naked mole rat',
 'platypus',
 'platypus',
 'platypus',
 'zebra',
 'zebra',
 'platypus',
 'zebra',
 'naked mole rat',
 'zebra',
 'naked mole rat',
 'giraffe',
 'platypus',
 'zebra',
 'naked mole rat',
 'giraffe',
 'giraffe',
 'zebra',
 'zebra',
 'giraffe',
 'platypus',
 'platypus',
 'naked mole rat',
 'platypus',
 'giraffe',
 'giraffe',
 'giraffe',
 'platypus',
 'naked mole rat',
 'platypus',
 'naked mole rat',
 'platypus',
 'giraffe',
 'giraffe',
 'zebra',
 'platypus',
 'platypus',
 'zebra',
 'giraffe',
 'platypus',
 'zebra',
 'platypus',
 'naked mole rat',
 'giraffe',
 'naked mole rat',
 'giraffe',
 'zebra',
 'giraffe',
 'giraffe',
 'zebra',
 'giraffe',
 'platypus',
 'zebra',
 'zebra',
 'zebra',
 'naked mole rat',
 'naked mole rat',
 'giraffe',
 'zebra',
 'giraffe',
 'platypus',
 'zebra',
 'platypus',
 'giraffe',
 'naked mole rat',
 'giraffe',
 'platypus',
 'zebra',
 'giraffe',
 'zebra',
 

In [113]:
#falta la declaracion, se ha borrado. ver del profesorheadcount={}
headcount={}
for criter in headcount:
    if criter in headcount:
        headcount[criter]+=1
    else: 
        headcount[criter]=1

headcount

#solucion: {'giraffe': 24, 'naked mole rat': 20, 'platypus': 27, 'zebra': 29}

{'giraffe': 24, 'naked mole rat': 20, 'platypus': 27, 'zebra': 29}

### (1d.1) Simple approach

 * Start with an empty dict
 * If a new key is not present in the dict, create it.
 * Otherwise, increase the frequency of the key by one.

In [118]:
#descrito en la celda 113

### (1d.2) Map/reduce

* Recall that *reduce* applies an operation to 2 elements of the same type, and returns another element of that type. Thus, the first thing to do is to map our collection to the type of the output. 
 
* The final result will be a dictionary of words in the vocabulary and counts. Therefore, we need to map each input word to this type.
 
* Then, we have to define a function, the combiner, that combines these dictionaries two at a time.

In [129]:
#combiner que sea capaz de combinar diccionarios
def combiner(left,right):
    
    return None

reduce(combiner,map(lambda bicho:{bicho,1},my_absurd_zoo))


In [130]:
#escribiendo los test. De esta forma es que formaliza los requerimientos.
{'giraffe':2}
{'cat':1}
{'dog':5, 'cat':4}
{'cow':0}
#esto es para afirmar: assert. Probamos y testeamos que la funcion hace lo que queramos
assert(combiner({'giraffe':2},{'dog':5, 'cat':4})=={'giraffe':2,'dog':5, 'cat':4})
assert(combiner({'giraffe':2,'cow':8},{'cow':0})=={'giraffe':2,'cow':8})

AssertionError: 

In [136]:
#combiner que sea capaz de combinar diccionarios
def combiner(left,right):
    
    for key in right:
        if key in left:
            left[key]+=right[key]
        else:
            left[key]=right[key]
    
    return left

reduce(combiner,map(lambda bicho:{bicho: 1}, my_absurd_zoo))

{'giraffe': 24, 'naked mole rat': 20, 'platypus': 27, 'zebra': 29}

# Part 2: Spark. Work with RDD and Pair RDD abstractions 

![drawing](https://prateekvjoshi.files.wordpress.com/2015/10/1-main4.png)

In [ ]:
#Para esta parte, hay que matar el proceso del terminal y poner en el shell: pyspark

# ** Apache Spark**

Apache Spark is an open source cluster computing framework originally developed in the AMPLab at University of California, Berkeley but was later donated to the Apache Software Foundation where it remains today. In contrast to Hadoop's two-stage disk-based MapReduce paradigm, Spark's multi-stage in-memory primitives provides performance up to 100 times faster for certain applications.

![](http://image.slidesharecdn.com/sparkandshark-120620130508-phpapp01/95/spark-and-shark-8-728.jpg?cb=1340197567)

By allowing user programs to load data into a cluster's memory and query it repeatedly, Spark is well-suited to machine learning algorithms.
![](http://spark.apache.org/images/logistic-regression.png)

Spark comes with a number of components that provide flexibility and generality.

<img src="http://spark.apache.org/images/spark-stack.png" alt="Drawing" style="width: 500px;"/>


## In this part, we keep on working on the word-count example, this time with spark. The basic abstraction of Spark is the Resilient Distributed Dataset (RDD):

#### «RDDs are fault-tolerant, parallel data structures that let users explicitly persist intermediate results in memory, control their partitioning to optimize data placement, and manipulate them using a rich set of operators.»

 * Read only, partitioned collection of records (immutable).
 * Stores the transformations used to build a dataset (its lineage), instead of the data itself. This property ensures fault-tolerance.
 * Users can control partitioning and persistence (caching).
 * RDDs are statically typed.
 * … and yes, everything is written in scala ;p. So you could use learning a bit of it!
 
<img src="http://eng.trueaccord.com/wp-content/uploads/2014/10/scala-logo.png" alt="Drawing" style="width: 200px;"/>

#### We will be trying to understand this abstraction with simple examples, using the [Python API](http://spark.apache.org/docs/latest/api/python/index.html)




### ** (2a) Create a base RDD: parallelize, actions and transformations **
We'll start by generating a base RDD by using a Python list and the `sc.parallelize` method.  Then we'll print out the type of the base RDD.

We use sc.parallelize to convert a standard Python collection into an RDD.

In [1]:
absurd_animals=['platypus','zebra','giraffe','naked mole rat','star-nosed mole']

import random
random.seed(42)

zoo=[random.choice(absurd_animals) for _ in range(100)]

In Spark, `map` is a method of the RDD type.

In [2]:
#vamos a parelelizar zoo. 
#sc es el spark context y es nuestro punto de entrada a spark. Lo tenemos pq hemos inidiado con pyspark

In [2]:
sc

''

In [3]:
#nuestro primer rdd. Paralelize, distribuye la lista zoo por todo el cluster.
zoo_rdd=sc.parallelize(zoo)

#en spark la funcion map es un método de los rdd

NameError: name 'sc' is not defined

In [6]:
zoo_rdd

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:480

In [7]:
plurals=zoo_rdd.map(lambda animal: animal+'s')
plurals

PythonRDD[1] at RDD at PythonRDD.scala:48

In [ ]:
#no se va a ejecutar nada hasta que llevemos acabo una accion (lazy)--> por ejemplo reduce, collect, count, take...


**Nothing has actually happened!**

`parallellize` tells spark to distribute the data, but this is not actually done until we perform some action.

Possible actions include counting, collecting, reducing, taking, etc. Take a look at the [Spark programming guide](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#actions)


In [9]:
plurals.take(5)
# hace solo el map para los 5, no para todos 

['platypuss', 'platypuss', 'giraffes', 'zebras', 'zebras']

In [10]:
all_plurals=plurals.collect()
len(all_plurals)

100

In [12]:
zoo_rdd.reduce()

100

#### Exercise

Calculate, from the rdd generated from list_of_cakes, how many cakes we have whose name does **not** start with an 's'

In [22]:
#actions(count) y transformations(filter)

def startwithS(word):
    if word[0]!='s':
        return True
    return False

zoo_rdd.filter(startwithS).count()

84

In [20]:
#otra forma con lambda: 
zoo_rdd.filter(lambda word: word[0]!='s').count()

84

In [23]:
#Podiamos haber hecho esto para una lista de muchos elementos y no para una lisa de 100

A very common way to write these chained operations in Spark is to put each on a line. Due to Python's syntax, we need to escape the newline character with a backslash.

In [27]:
zoo_rdd.filter(lambda word: word[0]!='s')\
    .count()
    
#con \ indicamos a python que ignore el salto de linea

84

### Actions and Transformations

There are two main types of operations in Spark:
[Actions](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#actions) and [Transformations](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#transformations).

**Transformations** produce an RDD. Some of the most important are `map`, `filter`, and `reduceByKey`. `sc.parallelize` and `sc.textFile` are not technically Transformations but you can think of them as such in one very important respect: they are *lazyly evaluated*. That is, when we perform a Transformation, we are only describing the operation to be performed at some point in the future. They attach a node to the execution graph.

**Actions** are implemented as methods on an RDD, and return an object of a type **that is not an RDD**. When we perform an Action, we give the order to execute the previously described transformations: we trigger the execution of the graph. Some of the most important are `reduce`, `collect`, `take`, `takeOrdered`, and `count`.

In [ ]:
#todos los tipos de operaciones en spark son o actions o transformations.

#Estas ultimas, generan un RDD. Son siempre lazy
#las actions se ejecutan sobre un RDD y devuelven algo que no es un RDD, dan la señal para que se haga el procesamiento (se ejecuta todo el grafo)
#Se ejecuta todo el grafo con el action. Aquí es donde se puede introducir el concepto de cachear (guardar en memoria)

# Cuando hacemos transformacione estamosdeifiniendo el procesado que sufrirán nuestros datos en algun momento.

In [31]:
import math

In [38]:
%%time
rdd=sc.parallelize(range(100000000))

CPU times: user 2.12 ms, sys: 467 µs, total: 2.59 ms
Wall time: 7 ms


In [42]:
%%time
roots=rdd.map(math.sqrt)

CPU times: user 13 µs, sys: 3 µs, total: 16 µs
Wall time: 18.8 µs


In [47]:
%%time
roots.reduce(lambda x, y:x+y) 

CPU times: user 6.59 ms, sys: 451 µs, total: 7.04 ms
Wall time: 9.56 s


666666661666.484

### **(2b) Persisting and the RDD lineage**

So far, we have seen that Spark RDDs are *lazyly evaluated*, i.e. nothing is actually done until an action is performed. In the RDD, the set of transformations to be applied are remembered: this is known as its *lineage*. It has the important consequence of making Spark RDDs *fault tolerant* automatically.

![](http://images.slideplayer.com/14/4499833/slides/slide_10.jpg) 

It might be interesting to store some intermediate results, though: perhaps because we want to apply several different transformations starting from that point, or because we are going to apply an iterative computation (as is customary in machine learning algorithms). For this, Spark has [several ways of persisting](http://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence)

In [48]:
roots.cache()
#esto lo que le está diciendo es guardame el resultado, y no se genera el resultado en ese momento. 
#le dice, de ese, una vez que lo calcules, no te olvides de él.


PythonRDD[19] at RDD at PythonRDD.scala:48

In [51]:
#la siguiente vevz que lo ejecute tardará menos. Vemos el proceso correctamente con la sintaxis correcta
roots=sc.parallelize(range(100000000)).map(math.sqrt).cache()

In [52]:
% time roots.map(lambda a: -a).take(5)

CPU times: user 4.5 ms, sys: 4.5 ms, total: 9 ms
Wall time: 6.81 s


[-0.0, -1.0, -1.4142135623730951, -1.7320508075688772, -2.0]

In [ ]:
#veremos esto mejor mas hacia delante pq ahora no funciona al profesor.

In [53]:
from pyspark import StorageLevel #nos permite indicar distintos niveles de almacenamiento (si queremos cambiarlo hay que llamar a unpersist)

StorageLevel cannot be changed after it has been set. We need to `unpersist` first.

In [55]:
roots.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [56]:
roots.cache()
roots.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [59]:
help(StorageLevel)

Help on class StorageLevel in module pyspark.storagelevel:

class StorageLevel(builtins.object)
 |  Flags for controlling the storage of an RDD. Each StorageLevel records whether to use memory,
 |  whether to drop the RDD to disk if it falls out of memory, whether to keep the data in memory
 |  in a JAVA-specific serialized format, and whether to replicate the RDD partitions on multiple
 |  nodes. Also contains static constants for some commonly used storage levels, MEMORY_ONLY.
 |  Since the data is always serialized on the Python side, all the constants use the serialized
 |  formats.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, useDisk, useMemory, useOffHeap, deserialized, replication=1)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  

In [ ]:
#StorageLevel() sirve para marcar un RDD
StorageLevel()


In [ ]:
#ver por parte del profesor...

### **(2c) Partitioning **

One important parameter for parallel collections is the number of partitions to cut the dataset into. Spark will run one task for each partition of the cluster. Typically you want 2-4 partitions for each CPU in your cluster.

To get the number of partitions of an RDD, just use `getNumPartitions()` on your RDD. You can change the partitions during RDD creation (with `parallelize(collection,numPartitions)` or `fromTextFile(file,numPartitions)`), or afterwards with methods like `repartition(), coalesce()`, etc.

In [3]:
#las particiones son los trozos, partes en los que nuestro dataset ha sido dividido. Cada partición está en un nodo.
#podemos saber el nº de particiones en las que esta dividido:
zoo_rdd.getNumPartitions()

2

In [4]:
#tambien podemos determinar el nº de particiones
zoo_rdd=sc.parallelize(zoo,numSlices=10)
zoo_rdd.getNumPartitions()

10

In [ ]:
#tambien podemos modificar el nº de particiones sobre la marcha. 
#Un aspecto muy importante  cuando trabajamos con Spark es que los RDD son inmutables, no se pueden modificar.
#coalerse() decrementa el nº de particiones de un RDD, y se trata de una transmision pq devuelve un RDD.
#repartitions(), mezcla los datos en un RDD nuevo en el que puede decrementar o incrementar el nº de particiones. (lo vamos a evitar usar)

We can see the partitions using [glom()](http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=glom#pyspark.RDD.glom): it retruns an RDD created by coalescing all elements within each partition into a list.

In [5]:
#glom() devuelve un Rdd poniendo todos los elementos juntos en una lista. (devuelve un RDD de listas)
glommed=zoo_rdd.glom()
glommed

PythonRDD[2] at RDD at PythonRDD.scala:48

In [6]:
#Podemos echarle un ojo haciendo un collect()
glommed.collect()

[['platypus',
  'platypus',
  'giraffe',
  'zebra',
  'zebra',
  'zebra',
  'platypus',
  'star-nosed mole',
  'platypus',
  'star-nosed mole'],
 ['naked mole rat',
  'platypus',
  'platypus',
  'platypus',
  'zebra',
  'zebra',
  'star-nosed mole',
  'star-nosed mole',
  'platypus',
  'star-nosed mole'],
 ['zebra',
  'star-nosed mole',
  'naked mole rat',
  'zebra',
  'naked mole rat',
  'star-nosed mole',
  'giraffe',
  'platypus',
  'zebra',
  'naked mole rat'],
 ['giraffe',
  'giraffe',
  'zebra',
  'zebra',
  'giraffe',
  'platypus',
  'platypus',
  'naked mole rat',
  'platypus',
  'giraffe'],
 ['giraffe',
  'star-nosed mole',
  'giraffe',
  'platypus',
  'naked mole rat',
  'star-nosed mole',
  'platypus',
  'naked mole rat',
  'platypus',
  'star-nosed mole'],
 ['giraffe',
  'star-nosed mole',
  'giraffe',
  'star-nosed mole',
  'zebra',
  'platypus',
  'platypus',
  'zebra',
  'giraffe',
  'platypus'],
 ['zebra',
  'platypus',
  'naked mole rat',
  'giraffe',
  'naked mole rat

In [8]:
#nos quedamos solo con las cebras. Va a tener las mismas parciciones que su papá
zebras=zoo_rdd.filter(lambda animal:animal=='zebra')
zebras.getNumPartitions()

10

In [9]:
#el filter se aplica sobre cada parción independientemente
zebras.glom().collect()

[['zebra', 'zebra', 'zebra'],
 ['zebra', 'zebra'],
 ['zebra', 'zebra', 'zebra'],
 ['zebra', 'zebra'],
 [],
 ['zebra', 'zebra'],
 ['zebra', 'zebra', 'zebra'],
 ['zebra', 'zebra', 'zebra'],
 ['zebra', 'zebra'],
 ['zebra', 'zebra', 'zebra', 'zebra']]

In [ ]:
#lo que está haciendo es que el procesado se hace en cada partición. Si zebra no está en una partición, su lista esta aparecerá vacía para esa partición. 

Partitions are one of the most powerfull concepts in Spark: you can decide how to distribute your data so it can fit in memory, and more importantly, you can perform computations on each partition *before* speaking to other partitions. This can have an enormous impact on performance

### **(2c) Pair RDDs: *grouping* strategies in Spark**

The next step in writing our word counting program is to create a new type of RDD, called a pair RDD. A pair RDD is an RDD where each element is a pair tuple (k, v) where k is the key and v is the value. In this example, we will create a pair consisting of ('<word>', 1) for each word element in the RDD, as we did in the map/reduce version of the histogram in Python, section (1d.2).

We can create the pair RDD using the map() transformation with a lambda() function to create a new RDD.

In [11]:
#cuando es un rdd de tuplas, el primer elemento se toma como key y el segundo como value.
#la key se interpreta como la identidad de ese elemento. Esto nos puede recordar a un diccionario con la diferencia que un diccionario solo tiene una entrada por cada key.

zoo_rdd.take(5)

['platypus', 'platypus', 'giraffe', 'zebra', 'zebra']

In [12]:
#cogemos el wordcount que definimos antes
def dictcombiner(left,right):
    
    for key in right:
        if key in left:
            left[key]+=right[key]
        else:
            left[key]=right[key]
    
    return left

In [13]:
zoo_rdd.map(lambda animal:{animal:1}).reduce(dictcombiner)

{'giraffe': 22,
 'naked mole rat': 15,
 'platypus': 23,
 'star-nosed mole': 16,
 'zebra': 24}

In [18]:
#ahora esto es completamente paralelizable, podemos aplicarlo a cualquier lista de palabras. Su potencial es completamente mayor
zoo_rdd.count()

100

In [ ]:
#esto es tan comun que spark nos pone una serie de métodos en los RDD que facilitan esto: gruopByKey()

In [19]:
tuples=zoo_rdd.map(lambda animal: (animal,1))
tuples.take(5)

[('platypus', 1), ('platypus', 1), ('giraffe', 1), ('zebra', 1), ('zebra', 1)]

### ** (2c.1) `groupByKey()` approach **
An approach you might first consider (we'll see shortly that there are better ways) is based on using the [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) transformation. As the name implies, the `groupByKey()` transformation groups all the elements of the RDD with the same key into a single list in one of the partitions. There are two problems with using `groupByKey()`:
  + The operation requires a lot of data movement to move all the values into the appropriate partitions.
  + The lists can be very large. Consider a word count of English Wikipedia: the lists for common words (e.g., the, a, etc.) would be huge and could exhaust the available memory in a worker.
 
Use `groupByKey()` to generate a pair RDD of type `('word', iterator)`. Next, sum the iterator using a `map()` transformation.  The result should be a pair RDD consisting of (word, count) pairs.

In [21]:
#queremos poner todos los tipos de animales 
groups=tuples.groupByKey()
groups.collect()

[('zebra', <pyspark.resultiterable.ResultIterable at 0x7f63ac2c87b8>),
 ('giraffe', <pyspark.resultiterable.ResultIterable at 0x7f63ac2c84a8>),
 ('platypus', <pyspark.resultiterable.ResultIterable at 0x7f63ac2c8b38>),
 ('star-nosed mole',
  <pyspark.resultiterable.ResultIterable at 0x7f63ac2c8828>),
 ('naked mole rat', <pyspark.resultiterable.ResultIterable at 0x7f63ac2c85c0>)]

In [22]:
#vemos los elementos decada key (los value) con un map
#si hacemos solo un count() no dá el nº de grupos
groups.count()

5

In [23]:
groups.map(lambda result_tuple: (result_tuple[0],len(result_tuple[1]))).collect()

[('zebra', 24),
 ('giraffe', 22),
 ('platypus', 23),
 ('star-nosed mole', 16),
 ('naked mole rat', 15)]

In [ ]:
#puede ser que una lista de values de una key sea de mucho tamaño, y puede hacer que el nodo(ordenador=worker) pete por falta de memoria
#otro problema es que hace uso de una transferencia por la red de manera innecesaria

#me evito estos problemas cuanta más agregacion pueda antes de enviar nada. como? --> reduceByKey().
#en vez de tomar todos los valores del RDD directamente, se procesan en carriles antes de transmtir.

### ** (2c.2)  `reduceByKey` approach **
A better approach is to start from the pair RDD and then use the [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) transformation to create a new pair RDD. 

The `reduceByKey()` transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value. `reduceByKey()` operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.

![](https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/images/reduce_by.png)

In [24]:
help(tuples.reduceByKey)

Help on method reduceByKey in module pyspark.rdd:

reduceByKey(func, numPartitions=None, partitionFunc=<function portable_hash at 0x7f63bc1c4598>) method of pyspark.rdd.PipelinedRDD instance
    Merge the values for each key using an associative and commutative reduce function.
    
    This will also perform the merging locally on each mapper before
    sending results to a reducer, similarly to a "combiner" in MapReduce.
    
    Output will be partitioned with C{numPartitions} partitions, or
    the default parallelism level if C{numPartitions} is not specified.
    Default partitioner is hash-partition.
    
    >>> from operator import add
    >>> rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
    >>> sorted(rdd.reduceByKey(add).collect())
    [('a', 2), ('b', 1)]



In [25]:
tuples.take(5)

[('platypus', 1), ('platypus', 1), ('giraffe', 1), ('zebra', 1), ('zebra', 1)]

In [27]:
#reduceByKey usa un combiner, tomará los values de las tuplas, es decir enteros y excupirá enteros
#hace un grupbykey y luego hace un reduce
#y como vemos reduceByKey es un 
count=tuples.reduceByKey(lambda x,y:x+y)
count

#el resultado de un reducebykey produce una coleccion de elementos, tan grande como el vocabulario del corpus. Por eso dejan la opcion de decidir tu en qué momento quieres que se materialice.
#también es porque pudedes seguir haciendo joins y demás operaciones.

PythonRDD[34] at RDD at PythonRDD.scala:48

In [29]:
count.collect()

[('zebra', 24),
 ('giraffe', 22),
 ('platypus', 23),
 ('star-nosed mole', 16),
 ('naked mole rat', 15)]

#### Summary: word count in Spark:

In [30]:
zoo_rdd.map(lambda animal: (animal,1))\
    .reduceByKey(lambda x,y:x+y)\
    .collect()

[('zebra', 24),
 ('giraffe', 22),
 ('platypus', 23),
 ('star-nosed mole', 16),
 ('naked mole rat', 15)]

## (2d) Apply word count to a file

### ** (2d.1) Load a text file **
For the next part of this lab, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into an RDD, we use the `SparkContext.textFile()` method. We also apply the recently defined `removePunctuation()` function using a `map()` transformation to strip out the punctuation and change all text to lowercase.  Since the file is large we use `take(15)`, so that we only print(15 lines.)

In [32]:
!wget http://www.gutenberg.org/files/100/100-0.txt -O shakespeare.txt

--2018-10-06 10:12:51--  http://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5852404 (5,6M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   5,58M  1,27MB/s    in 4,4s    

2018-10-06 10:12:56 (1,27 MB/s) - ‘shakespeare.txt’ saved [5852404/5852404]



In [33]:
#obras completas de Shakespeare
!head shakespeare.txt

In [34]:
#para leer el archivo, primero vamos a abrirlo
shakespeare=sc.textFile('shakespeare.txt')
shakespeare.take(5)
#es un rdd donde cada elemento es una linea del fichero.

['',
 'Project Gutenberg’s The Complete Works of William Shakespeare, by William',
 'Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere in the United States and']

In [ ]:
#queremos aplicar a este corpus el workcount
#primero, vamos a eleminar las mayus y la puntuación

### ** (2d.2) Capitalization and punctuation **
Real world files are more complicated than the data we have been using in this lab. Some of the issues we have to address are:
  + Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
  + All punctuation should be removed.
  + Any leading or trailing spaces on a line should be removed.
 
Define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces.  Use the Python [re](https://docs.python.org/2/library/re.html) module to remove any text that is not a letter, number, or space. Reading `help(re.sub)` might be useful.

In [35]:
#eliminación de las mayusculas
lowercased=shakespeare.map(str.lower)
lowercased.take(5)
#podíamos haberlo hecho con una lambda tambien

['',
 'project gutenberg’s the complete works of william shakespeare, by william',
 'shakespeare',
 '',
 'this ebook is for the use of anyone anywhere in the united states and']

In [36]:
#para quitar signos de puntuacion vamos a definir una funcion y a aplicarla con un map
#usaremos re.sub
import re
help(re.sub) 
#es una funcion que nos permite sustituir en funcion de las expresiones regulares.

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the match object and must return
    a replacement string to be used.



In [45]:
def remove_punctuation(line):
    return re.sub('[^\w ]', '', line)#para denotar todo lo que no sean palabras, eso lo que denota esa expresion regular

In [46]:
#testing
remove_punctuation('askdjfas.asdf.as.df.amasdfasm,  ')

'askdjfasasdfasdfamasdfasm  '

In [47]:
clean_lines=lowercased.map(remove_punctuation)
clean_lines.take(15)

['',
 'project gutenbergs the complete works of william shakespeare by william',
 'shakespeare',
 '',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever  you may copy it give it away or reuse it under the terms',
 'of the project gutenberg license included with this ebook or online at',
 'wwwgutenbergorg  if you are not located in the united states youll',
 'have to check the laws of the country where you are located before using',
 'this ebook',
 '',
 'see at the end of this file  content note added in 2017 ',
 '',
 '']

In [ ]:
#tenemos las lineas ya, tenemos que dividir en palabras

http://www.regular-expressions.info/quickstart.html

https://regex101.com/

### ** (2d.3) Words from lines **
Before we can use the `wordcount()` function, we have to address two issues with the format of the RDD:
  + The first issue is that  that we need to split each line by its spaces.
  + The second issue is we need to filter out empty lines.
 
Apply a transformation that will split each element of the RDD by its spaces. For each element of the RDD, you should apply Python's string [split()](https://docs.python.org/3/library/stdtypes.html#str.split) function. You might think that a `map()` transformation is the way to do this, but think about what the result of the `split()` function will be.

Check out the Spark Programming Guide for an alternative transformation that helps us here.

In [48]:
remove_punctuation('askdjfas.asdf.as.df.amasdfasm,  ').split()

['askdjfasasdfasdfamasdfasm']

In [52]:
#tenemos que mapear un split() sobre clean lines
words=clean_lines.map(str.split)
words.take(5)
#ahora es un rdd de listas, antes era de strings

[[],
 ['project',
  'gutenbergs',
  'the',
  'complete',
  'works',
  'of',
  'william',
  'shakespeare',
  'by',
  'william'],
 ['shakespeare'],
 [],
 ['this',
  'ebook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'united',
  'states',
  'and']]

In [57]:
#buscar una trsnformation en la guia de spark que nos produce lo que queremos.
words=clean_lines.flatMap(str.split)
words.take(10)
#esto ya nos da lo que queremos

['project',
 'gutenbergs',
 'the',
 'complete',
 'works',
 'of',
 'william',
 'shakespeare',
 'by',
 'william']

### ** (2d.4) Remove empty elements **
The next step is to filter out the empty elements.  Remove all entries where the word is `''`.

In [ ]:
#ya está hecho con flatMap

### (2d.5) Count the words and show the top 15

We know the drill at this point, don't we? We map to a tuple then `reduceByKey`

We can view the top 15 words by using the `takeOrdered()` action; however, since the elements of the RDD are pair tuples, we need a custom sort function that sorts using the value part of the pair rather than the key.

You'll notice that many of the words are common English words (know as stopwords).

Use our map reduce and and `takeOrdered()` to obtain the fifteen most common words and their counts.

In [67]:
totalcounts=words.map(lambda word: (word,1))\
    .reduceByKey(lambda x,y:x+y)
    
totalcounts.take(5)


[('project', 107),
 ('of', 18815),
 ('shakespeare', 26),
 ('this', 7185),
 ('ebook', 14)]

In [68]:
#takeOrdered()
totalcounts.takeOrdered(15, lambda element: -element[1])
# queremos ordenar en base al nº de apariciones descendentemente, de ahi el -element[1]

[('the', 30002),
 ('and', 28358),
 ('i', 21867),
 ('to', 20816),
 ('of', 18815),
 ('a', 15992),
 ('you', 14437),
 ('my', 13191),
 ('in', 12032),
 ('that', 11781),
 ('is', 9709),
 ('not', 9067),
 ('with', 8518),
 ('me', 8270),
 ('for', 8187)]

# Practical

## ETL with airline coupon data

Load the data first: coupon data

In [1]:
data=sc.textFile('coupon150720.csv')
data.take(5)
#textFile me devuelve siempre un RDD de strings.
#Representan cupones de viajes aéreos en aerolineas. Un cupón era lo que se intercambiaba en los aeropuertos por las tarjetas de embarque.
#los cupones representan trayectos: MAD-LON-MUN son 2 trayectos, luego se gastan 2 cupones.
#la primera columna es el nº de ticket


['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 '79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 '79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 '79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [ ]:
#Si tenemos errores del tipo 'name 'sc' is not defined', hacer en el terminal: rm -rf metastore_db/ y reiniciar el kernel

In [6]:
#!rm -rf metastore_db/

#### Exercise

Take fields 0, 2, 3, 4, and 6 from each line of coupons

In [2]:
def parse(line):
    
    fields=line.split(',')
    coupon=[fields[0],fields[2],fields[3],fields[4],float(fields[6])]
    
    return coupon

testline='79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'
parse(testline)

['79062005698500', 'MAA', 'AUH', '9W', 56.79]

In [3]:
#una vez testeada y funcionando aplicamos a todo con map
coupons=data.map(parse)
coupons.take(5)

[['79062005698500', 'MAA', 'AUH', '9W', 56.79],
 ['79062005698500', 'AUH', 'CDG', '9W', 84.34],
 ['79062005924069', 'CJB', 'MAA', '9W', 60.0],
 ['79065668570385', 'DEL', 'DXB', '9W', 160.63],
 ['79065668737021', 'AUH', 'IXE', '9W', 152.46]]

#### Intermission: Stack traces in Spark

Stack traces in pyspark are quite convoluted because we have many layers of processing. Look for the python stack trace contained within the Py4JJavaError message.

#### Exercise

Keep only the amount. Get average, max, min and std

In [4]:
amounts=coupons.map(lambda coupon: coupon[4])
amounts.take(5)

[56.79, 84.34, 60.0, 160.63, 152.46]

In [7]:
min_=amounts.reduce(lambda x,y: x if x<y else y)
max_=amounts.reduce(lambda x,y: x if x>y else y)         

min_, max_
#formas
#min_=amount.min()
#max_=amount.max()   

(0.0, 6355194.0)

In [11]:
#usaremos la funcion zip: hace algo parecido a transponer

def combiner_tuples(tuple1,tuple2):
    #entrelazando tupla 1 y tupla 2
    
    #esto es una tuple comprenhension
    #se trata de un combiner que nos suma los elementos de las tuplas, independientemente del tipo de elementos que tengamos
    #son lazy tambien, hay que forzar la ejecucion en este caso.
    return tuple(a+b for a,b in zip(tuple1,tuple2))

In [13]:
sum_squares, total, count=amounts.map(lambda n: (n**2,n,1))\
                            .reduce(combiner_tuples)
sum_squares, total, count

(122763999131054.75, 184831898.49994844, 1232662)

In [15]:
mean=(total/count)
std=((sum_squares-count*mean**2)/(count-1))**0.5

mean,std

(149.94532037164157, 9978.486133659524)

#### All in one go!



In [17]:
#modificando para que la funcoin haga el max y el minimo
#necesitamos una tupla de 5.1º para el minimo, 2º para el maximo, 3º pra el cuadrado... y asi como antes

tuples=amounts.map(lambda amount: (amount, amount, amount**2, amount, 1))
tuples.take(5)

[(56.79, 56.79, 3225.1041, 56.79, 1),
 (84.34, 84.34, 7113.235600000001, 84.34, 1),
 (60.0, 60.0, 3600.0, 60.0, 1),
 (160.63, 160.63, 25801.9969, 160.63, 1),
 (152.46, 152.46, 23244.051600000003, 152.46, 1)]

In [18]:
#test

#tenemos que modificar que la operacion no siempre no sea sumar. Tenemos que hacer una operacion distinta para combinar cada elemento de la tupla 
#podemos hace una lista de funciones. Todo son objetos en python, podemos meterlas en listas... etc.
#en este caso esta es la lista de funciones que tenemos que usar
#[min,max,lambda x,y:x+y,lambda x,y:x+y,lambda x,y:x+y]
#otra forma es con add

from operator import add
add(1,2)

3

In [19]:
ops=[max,min,add,add,add]

In [108]:
#pasamos la lista de funciones como argumento y apmliamos el zip para seleccionar la operacion
#tenemos que desempaquetar ahora 3 elementos.
def combiner_tuples(tuple1,tuple2,ops):
    return tuple(op(a+b) for op,a,b in zip(ops,tuple1,tuple2))

sample1=(56.79, 56.79, 3225.1041, 56.79, 1)
sample2=(84.34, 84.34, 7113.235600000001, 84.34, 1)

combiner_tuples(sample1,sample2,ops)

#ver codigo profesor

TypeError: 'float' object is not iterable

In [23]:
result=amounts.map(lambda amount:(amount,amount,amount**2,amount,1))\
                .reduce(lambda t1, t2: combiner_tuples(t1,t2,ops))
    
#ver codigo profesor

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 28, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 833, in func
    yield reduce(f, iterator, initial)
  File "<ipython-input-23-7943910dad44>", line 1, in <lambda>
  File "<ipython-input-22-77fcbcd8f9d3>", line 4, in c

#### Exercise

Get stats on ticket amount for all tickets with destination MAD

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

#### Part 1

We need to filter coupons with destination Madrid, and after that group on the origin. 

In [62]:
destMAD=coupons.filter(lambda dest: dest[2]=='MAD')
destMAD.take(5)
abreviado=destMAD.map(lambda fila: (fila[1],fila[4]))
abreviado.take(5)

[('BRU', 21.02), ('BRU', 27.66), ('CDG', 46.97), ('CDG', 3.38), ('CDG', 26.02)]

In [65]:

result=abreviado.groupByKey().map(lambda tuple_: (tuple_[0],sum(tuple_[1])))
result.take(5)


[('BRU', 21232.430000000008),
 ('TUN', 882.5100000000001),
 ('AMS', 29872.269999999953),
 ('VGO', 11697.699999999997),
 ('LCY', 10094.06)]

In [67]:
#con mapValues se toma soo el value de la tupla
result2=abreviado.groupByKey().mapValues(lambda value: sum(value))
result2.take(5)

[('BRU', 21232.430000000008),
 ('TUN', 882.5100000000001),
 ('AMS', 29872.269999999953),
 ('VGO', 11697.699999999997),
 ('LCY', 10094.06)]

In [ ]:
#result o result2 son el precio total por origen
#normalmente la unica acttion que se va a hacer ese en un spark real es save(). Collect se hace muy poco

In [68]:
result=abreviado.reduceByKey(lambda x,y:x+y)
result.take(5)

[('BRU', 21232.430000000008),
 ('TUN', 882.5100000000001),
 ('AMS', 29872.26999999995),
 ('VGO', 11697.699999999995),
 ('LCY', 10094.06)]

At this point, we only need the origin, to group on, and the value, to sum

We are now ready to reduce all amounts per origin.

#### Part 2

This is very similar, with two differences: we need to group on the airline, and to calculate averages we need to keep track of both the total amount on the coupons and the number of coupons.

In [98]:
aerolineas=destMAD.map(lambda fila: (fila[3],fila[4],1))
aerolineas.take(5)

[('UX', 21.02, 1),
 ('SN', 27.66, 1),
 ('AF', 46.97, 1),
 ('AF', 3.38, 1),
 ('AF', 26.02, 1)]

In [99]:
#reduce by key y gruop by key solo funcionan con tuplas de 2. Para transformar duplas de 3 en una tupla de 2 hay que empaquetar
amounts=destMAD.map(lambda fila: (fila[3],(fila[4],1)))
amounts.take(5)

[('UX', (21.02, 1)),
 ('SN', (27.66, 1)),
 ('AF', (46.97, 1)),
 ('AF', (3.38, 1)),
 ('AF', (26.02, 1))]

In [100]:
amounts.reduceByKey(lambda t1, t2: (t1[0]+t2[0], t1[1]+t2[1])).take(5)

[('SN', (3092.6099999999997, 61)),
 ('IB', (928396.8400000058, 7526)),
 ('KL', (18007.62999999999, 323)),
 ('AZ', (3715.28, 54)),
 ('', (0.0, 168))]

In [102]:
amounts_by_airline=amounts.reduceByKey(lambda t1, t2:combiner_tuples(t1,t2,[add,add]))
amounts_by_airline.take(5)
#no funciona pq la funcion combiner_tuples no funcoina correcatmente

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 89.0 failed 1 times, most recent failure: Lost task 1.0 in stage 89.0 (TID 146, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 346, in func
    return f(iterator)
  File "/usr/local/spark/python

In [103]:
averages_by_airline.map(lambda t: t[0], (t[1][0]/t[1][1]))
averages_by_airline.take(5)


NameError: name 'averages_by_airline' is not defined

In [104]:
#con map values
averages_by_airline.mapValues(lambda t: t[0]/t[1])
averages_by_airline.take(5)

NameError: name 'averages_by_airline' is not defined

In [ ]:
averages_by_airline.takeOrdered(10,)

Now we reduce tuples, summing each component of the tuple separately.

This reduceByKey generates an RDD of the form (k, (v1, v2)). We can map it like this:

Or we can use the mapValues method to only process the (v1, v2) part, ignoring the key for processing without discarding it.

And we're ready to take our result.

#### Stretch: 

Get the totals from first origin without using the coupon number

In [107]:
#necesitamos groupbykey, en lugar de reduceByKey  pq queremos el primer origen, independientemente de las escalas
destMAD.take(5)

[['79062005639642', 'BRU', 'MAD', 'UX', 21.02],
 ['79065668754871', 'BRU', 'MAD', 'SN', 27.66],
 ['79065668917696', 'CDG', 'MAD', 'AF', 46.97],
 ['79062006133090', 'CDG', 'MAD', 'AF', 3.38],
 ['79062006110497', 'CDG', 'MAD', 'AF', 26.02]]

We'll use groupBy this time, because we need to look at every coupon in a ticket in order to identify first origin.

A ResultIterable is a lazy collection, so we can take it and iterate over it, or turn it into a list to materialize it. This is useful to get a test case to test the function we will be writing.

We are going to compare the set of origins to the set of destinations. The one that is in the first but not in the second should be the first origin.

Now we have have our values, so we are ready to get the totals and sort. Before we reduceByKey, we need to reshape our tuples so the first origin is the key.

### Further resources

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_Cheat_Sheet_Python.pdf